In [1]:
!pip install -U beautifulsoup4

     |████████████████████████████████| 97 kB 3.4 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import sqlite3
import numpy as np
import pandas as pd
import requests
from lxml import etree
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

In [3]:
class IMDBScraper:
    def __init__(self):
        self.request_headers = {
            'accept-language': "en-US,en;q=1.0",
            'content-language': 'en-US',
            'user-agent': 'user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36'
        }
    def get_top_rated(self):
        response = requests.get('https://www.imdb.com/chart/top', headers=self.request_headers)
        soup = BeautifulSoup(response.text)
        movie_titles = [e.text for e in soup.select('.titleColumn a')]
        movie_years = [int(e.text.replace('(', '').replace(')', '')) for e in soup.select('.secondaryInfo')]
        top_rated_movies = pd.DataFrame()
        top_rated_movies['id'] = range(1, len(movie_titles) + 1)
        top_rated_movies['title'] = movie_titles
        top_rated_movies['release_year'] = movie_years
        movie_page_links = [e.get("href") for e in soup.select('.titleColumn a')]
        self.movie_titles = movie_titles
        self.movie_page_links = movie_page_links
        return top_rated_movies
    def get_movie_info(self):
        top_rated_movies = self.get_top_rated()
        movie_ids = top_rated_movies['id'].values
        movie_info = []
        for mplink, mid, mtitle in tqdm(zip(self.movie_page_links, movie_ids, self.movie_titles)):
            rating_page = f"https://www.imdb.com{mplink}ratings"
            response = requests.get(rating_page, headers=self.request_headers)
            soup = BeautifulSoup(response.text)
            try:
                rating = float(soup.select('.ipl-rating-star__rating')[0].text)
            except:
                rating = None
            movie_page = f"https://www.imdb.com{mplink}"
            response = requests.get(movie_page, headers=self.request_headers)
            soup = BeautifulSoup(response.text)
            try:
                director = soup.select('a.ipc-metadata-list-item__list-content-item--link')[0].text
            except:
                director = None
            soup = BeautifulSoup(response.content, "html.parser")
            dom = etree.HTML(str(soup))
            release_year = int(dom.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div[1]/div/ul/li[1]/a')[0].text)
            if mtitle == "The Boat":
                movie_time = soup.select("div > ul > li:nth-child(2)")[0].text.split()
            else:
                movie_time = soup.select("div > ul > li:nth-child(3)")[0].text.split()
            if len(movie_time) == 2:
                hours, mins = int(movie_time[0].replace("h", "")), int(movie_time[1].replace("m", ""))
                movie_time_mins = hours * 60 + mins
            else:
                hours_mins = movie_time[0]
                if 'h' in hours_mins:
                    hours = int(hours_mins.replace('h', ''))
                    movie_time_mins = hours * 60
                elif 'm' in hours_mins:
                    mins = int(hours_mins.replace('m', ''))
                    movie_time_mins = mins
            credits_page = f"https://www.imdb.com{mplink}fullcredits"
            response = requests.get(credits_page, headers=self.request_headers)
            soup = BeautifulSoup(response.text)
            actors = [e.text.strip() for e in soup.select('.primary_photo+ td a')]
            if len(actors) > 15:
                actors = actors[:15]
            ords = list(range(1, len(actors) + 1))
            print(f"{mtitle}: {actors}")
            movie_data = {
                'id': mid,
                'rating': rating,
                'director': director,
                'release_year': release_year,
                'runtime': movie_time_mins,
                'actor': actors,
                'ord': ords
            }
            movie_info.append(movie_data)
        self._movie_info = movie_info
        return movie_info
    def get_top_rated_movies(self):
        top_rated = self.get_top_rated()
        movie_info = self.get_movie_info()
        movie_info_list = []
        for item in movie_info:
            movie_data = {
                'id': item['id'],
                'rating': item['rating'],
                'director': item['director'],
                'runtime': item['runtime']
            }
            movie_info_list.append(movie_data)
        movie_info_df = pd.DataFrame(movie_info_list)
        top_rated_movies = pd.merge(top_rated, movie_info_df, left_on='id', right_on='id')
        return top_rated_movies[['id', 'title', 'release_year', 'rating', 'director', 'runtime']]
    def get_actors(self):
        movie_info = self._movie_info
        actors_list = []
        for item in tqdm(movie_info):
            actor = item['actor']
            actors_list += actor
        unique_actors = set(actors_list)
        ordered_actors = sorted(list(unique_actors))
        ids = list(range(1, len(ordered_actors) + 1))
        actors = pd.DataFrame()
        actors['id'] = ids
        actors['name'] = ordered_actors
        self._actors = actors
        return actors
    def get_casting(self):
        movie_info = self._movie_info
        actors = self._actors
        casting = pd.DataFrame()
        for item in tqdm(movie_info):
            movie_id = item['id']
            names = item['actor']
            ords = item['ord']
            df = pd.DataFrame()
            df['actor_name'] = names
            df['ord'] = ords
            df['movie_id'] = movie_id
            casting = casting.append(df)
        casting_merged = pd.merge(casting, actors, left_on='actor_name', right_on='name', how='left')
        casting_selected = casting_merged[['movie_id', 'id', 'ord']]
        casting_selected.columns = ['movie_id', 'actor_id', 'ord']
        return casting_selected

In [4]:
response = requests.get("https://www.imdb.com/title/tt0111161/fullcredits")
soup = BeautifulSoup(response.text)
actors = [e.text.strip() for e in soup.select('.primary_photo+ td a')]
actors

['Tim Robbins',
 'Morgan Freeman',
 'Bob Gunton',
 'William Sadler',
 'Clancy Brown',
 'Gil Bellows',
 'Mark Rolston',
 'James Whitmore',
 'Jeffrey DeMunn',
 'Larry Brandenburg',
 'Neil Giuntoli',
 'Brian Libby',
 'David Proval',
 'Joseph Ragno',
 'Jude Ciccolella',
 'Paul McCrane',
 'Renee Blaine',
 'Scott Mann',
 'John Horton',
 'Gordon Greene',
 'Alfonso Freeman',
 'Vincent Foster',
 'John E. Summers',
 'Frank Medrano',
 'Mack Miles',
 'Alan R. Kessler',
 'Morgan Lund',
 'Cornell Wallace',
 'Gary Lee Davis',
 'Neil Summers',
 'Ned Bellamy',
 'Joe Pecoraro',
 'Harold E. Cope Jr.',
 'Brian Delate',
 'Don McManus',
 'Donald Zinn',
 'Dorothy Silver',
 'Robert Haley',
 'Dana Snyder',
 'John D. Craig',
 'Ken Magee',
 'Eugene C. DePasquale',
 'Bill Bolender',
 'Ron Newell',
 'John R. Woodward',
 'Chuck Brauchler',
 'Dion Anderson',
 'Claire Slemmer',
 'James Kisicki',
 'Rohn Thomas',
 'Charlie Kearns',
 'Rob Reider',
 'Brian Brophy',
 'Paul Kennedy',
 'James Babson',
 'Dennis Baker',
 'Fre

In [5]:
imdb_scraper = IMDBScraper()
top_rated_movies = imdb_scraper.get_top_rated_movies()
actors = imdb_scraper.get_actors()
casting = imdb_scraper.get_casting()

1it [00:02,  2.28s/it]

The Shawshank Redemption: ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton', 'William Sadler', 'Clancy Brown', 'Gil Bellows', 'Mark Rolston', 'James Whitmore', 'Jeffrey DeMunn', 'Larry Brandenburg', 'Neil Giuntoli', 'Brian Libby', 'David Proval', 'Joseph Ragno', 'Jude Ciccolella']


2it [00:05,  2.77s/it]

The Godfather: ['Marlon Brando', 'Al Pacino', 'James Caan', 'Richard S. Castellano', 'Robert Duvall', 'Sterling Hayden', 'John Marley', 'Richard Conte', 'Al Lettieri', 'Diane Keaton', 'Abe Vigoda', 'Talia Shire', 'Gianni Russo', 'John Cazale', 'Rudy Bond']


3it [00:10,  3.73s/it]

...

249it [09:57,  2.30s/it]

The Handmaiden: ['Kim Tae-ri', 'Yong-nyeo Lee', 'Min-chae Yoo', 'Dong-hwi Lee', 'Kim Hae-sook', 'Kim Min-hee', 'Ha Jung-woo', 'Kyu-jung Lee', 'Si-eun Kim', 'Si-yeon Ha', 'Rina Takagi', 'Cho Jin-woong', 'Geun-hee Won', 'Jong-Dae Kim', 'Han-sun Jang']


250it [10:01,  2.40s/it]


Dances with Wolves: ['Kevin Costner', 'Mary McDonnell', 'Graham Greene', 'Rodney A. Grant', "Floyd 'Red Crow' Westerman", 'Tantoo Cardinal', 'Robert Pastorelli', 'Charles Rocket', 'Maury Chaykin', 'Jimmy Herman', 'Nathan Lee Chasing His Horse', 'Michael Spears', 'Jason R. Lone Hill', 'Tony Pierce', 'Doris Leader Charge']


100%|██████████| 250/250 [00:00<00:00, 336.29it/s]


In [6]:
top_rated_movies.head()

,id,title,release_year,rating,director,runtime
0,1,The Shawshank Redemption,1994,9.3,Frank Darabont,142
1,2,The Godfather,1972,9.2,Francis Ford Coppola,175
2,3,The Dark Knight,2008,9.1,Christopher Nolan,152
3,4,The Godfather: Part II,1974,9.0,Francis Ford Coppola,202
4,5,12 Angry Men,1957,9.0,Sidney Lumet,96


In [7]:
actors.head()

,id,name
0,1,Aamir Khan
1,2,Aaron Eckhart
2,3,Abbas-Ali Roomandi
3,4,Abbey Lee
4,5,Abbie Cornish


In [8]:
casting.head()

,movie_id,actor_id,ord
0,1,2940,1
1,1,2185,2
2,1,329,3
3,1,3128,4
4,1,549,5


In [9]:
con = sqlite3.connect('imdb.db')
top_rated_movies.to_sql('movies', con, index=False)
actors.to_sql('actors', con, index=False)
casting.to_sql('casting', con, index=False)
cur = con.cursor()

In [10]:
create_movies = """
PRAGMA foreign_keys=off;
BEGIN TRANSACTION;
ALTER TABLE movies RENAME TO movies_no_keys;
CREATE TABLE movies (
    id INTEGER,
    title TEXT,
    release_year INTEGER,
    rating REAL,
    director TEXT,
    runtime int,
    PRIMARY KEY (id)
);
INSERT INTO movies SELECT * FROM movies_no_keys;
COMMIT;
PRAGMA foreign_keys=on;
"""
cur.executescript(create_movies)
con.commit()

In [11]:
create_actors = """
PRAGMA foreign_keys=off;
BEGIN TRANSACTION;
ALTER TABLE actors RENAME TO actors_no_keys;
CREATE TABLE actors (
    id INTEGER,
    name TEXT,
    PRIMARY KEY (id)
);
INSERT INTO actors SELECT * FROM actors_no_keys;
COMMIT;
PRAGMA foreign_keys=on;
"""
cur.executescript(create_actors)
con.commit()

In [12]:
create_casting = """
PRAGMA foreign_keys=off;
BEGIN TRANSACTION;
ALTER TABLE casting RENAME TO casting_no_keys;
CREATE TABLE casting (
    movie_id INTEGER,
    actor_id INTEGER,
    ord INTEGER,
    FOREIGN KEY (movie_id) REFERENCES movies (id) 
            ON DELETE CASCADE ON UPDATE NO ACTION
    FOREIGN KEY (actor_id) REFERENCES actors (id) 
            ON DELETE CASCADE ON UPDATE NO ACTION
);
INSERT INTO casting SELECT * FROM casting_no_keys;
COMMIT;
PRAGMA foreign_keys=on;
"""
cur.executescript(create_casting)
con.commit()

In [14]:
drop_tables = """
DROP TABLE movies_no_keys;
DROP TABLE actors_no_keys;
DROP TABLE casting_no_keys;
"""
cur.executescript(drop_tables)
con.commit()

In [15]:
con.close()